In [1]:
# Implementation of a simple Stack and an ArrayList with undo functionality.

class Stack:
    """A simple stack implementation using a linked list representation."""
    
    def __init__(self):
        # Initialize an empty stack with size 0 and no elements.
        self.inList = None
        self.size = 0
        
    def push(self, v):
        """Push a value onto the stack."""
        self.size += 1
        # Store the new value and link it to the current stack.
        self.inList = (v, self.inList)
        
    def pop(self):
        """Pop the top value from the stack."""
        # Assert ensures the stack is not empty before popping.
        if self.size == 0: assert(0)
        self.size -= 1
        # Retrieve the top value and update the stack pointer.
        (v, ls) = self.inList
        self.inList = ls
        return v
    
    def __str__(self):
        """Return a string representation of the stack."""
        s = "["
        ls = self.inList
        for _ in range(self.size):
            (v, ls) = ls
            s += str(v)
            if ls != None: s += ", "
        return s + "]"
        
class ArrayList:
    """A dynamic array-like data structure."""
    
    def __init__(self):
        # Initialize with a fixed capacity array and count of elements.
        self.inArray = [0 for _ in range(10)]
        self.count = 0
        
    def get(self, i):
        """Get the element at index i."""
        return self.inArray[i]

    def set(self, i, e):
        """Set the element at index i to e."""
        self.inArray[i] = e

    def length(self):
        """Return the current number of elements."""
        return self.count

    def append(self, e):
        """Add an element to the end of the array."""
        self.inArray[self.count] = e
        self.count += 1
        # Resize the array if capacity is reached.
        if len(self.inArray) == self.count:
            self._resizeUp()

    def insert(self, i, e):
        """Insert an element at index i, shifting subsequent elements."""
        for j in range(self.count, i, -1):
            self.inArray[j] = self.inArray[j-1]
        self.inArray[i] = e
        self.count += 1
        if len(self.inArray) == self.count:
            self._resizeUp()
    
    def remove(self, i):
        """Remove the element at index i and shift subsequent elements."""
        self.count -= 1
        val = self.inArray[i]
        for j in range(i, self.count):
            self.inArray[j] = self.inArray[j+1]
        return val

    def _resizeUp(self):
        """Double the array size when capacity is exceeded."""
        newArray = [0 for _ in range(2 * len(self.inArray))]
        for j in range(len(self.inArray)):
            newArray[j] = self.inArray[j]
        self.inArray = newArray
        
    def toArray(self):
        """Return a list representation of the current elements."""
        return self.inArray[:self.count]

    def __str__(self):
        """Return a string representation of the array."""
        if self.count == 0: return "[]"
        s = "["
        for i in range(self.count-1): s += str(self.inArray[i]) + ", "
        return s + str(self.inArray[self.count-1]) + "]"

class ArrayListWithUndo(ArrayList):
    """Extension of ArrayList to support undo operations."""
    
    def __init__(self):
        # Initialize ArrayList and a stack for undo operations.
        super().__init__()
        self.undos = Stack()

    def set(self, i, v):
        """Set element at index i and store undo information."""
        self.undos.push(("set", i, self.inArray[i]))
        self.inArray[i] = v

    def append(self, v):
        """Append an element and store undo information."""
        self.undos.push(("rem", self.count, None))
        super().append(v)

    def remove(self, i):
        """Remove element at index i and store undo information."""
        if i < self.count:  # Check if i is within bounds
            self.undos.push(("ins", i, self.inArray[i]))
            super().remove(i)

    def insert(self, i, v):
        """Insert element at index i and store undo information."""
        if i <= self.count:  # Ensure valid index
            self.undos.push(("rem", i, None))
            super().insert(i, v)

    def undo(self):
        """Undo the last operation."""
        if self.undos.size == 0:
            return

        op, i, v = self.undos.pop()

        if op == "set":
            super().set(i, v)  # Restore previous value
        elif op == "rem":
            if self.count == 0:
                return
            else:
                super().remove(i)  # Undo an append operation
        elif op == "ins":
            super().insert(i, v)  # Reinsert a removed element

    def __str__(self):
        """Return string representation of the list and undo stack."""
        return str(self.toArray()) + "\n-> " + str(self.undos)
    def __getitem__(self, index):
        return self.inArray[index]

    def __setitem__(self, index, value):
        self.inArray[index] = value

    def __len__(self):
        return len(self.inArray)


# Minimal tests for ArrayListWithUndo

def tprint(ls, i):
    """Print test case results."""
    print("\n=== Test", i, "===\n", ls, "\nsize", ls.length())

ls = ArrayListWithUndo()
A = [2, 3, 4, 5, 5, 1, 4]
for x in A: 
    ls.append(x)
ls.set(4, 2)
ls.insert(3, 10)
ls.remove(0)
tprint(ls, 0)
for i in range(len(A) + 4):
        ls.undo()
        tprint(ls, i + 1)



=== Test 0 ===
 [3, 4, 10, 5, 2, 1, 4]
-> [('ins', 0, 2), ('rem', 3, None), ('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 7

=== Test 1 ===
 [2, 3, 4, 10, 5, 2, 1, 4]
-> [('rem', 3, None), ('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 8

=== Test 2 ===
 [2, 3, 4, 5, 2, 1, 4]
-> [('set', 4, 5), ('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 7

=== Test 3 ===
 [2, 3, 4, 5, 5, 1, 4]
-> [('rem', 6, None), ('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 7

=== Test 4 ===
 [2, 3, 4, 5, 5, 1]
-> [('rem', 5, None), ('rem', 4, None), ('rem', 3, None), ('rem', 2, None), ('rem', 1, None), ('rem', 0, None)] 
size 6

=== Test 5 ===
 [2, 3, 4, 5, 5]
->

In [2]:
class Node:
 def __init__(self, d, n):
    self.data = d
    self.next = n

class linkedList:
    def __init__(self):
        self.head = None
        self.length = 0

    def search(self, d):
        i = 0
        ptr = self.head
        while ptr is not None:
            if ptr.data == d:
                return i
            ptr = ptr.next
            i += 1
        return -1

    def append(self, d):
        if self.head is None:
            self.head = Node(d, None)
        else:
            ptr = self.head
            while ptr.next is not None:
                ptr = ptr.next
            ptr.next = Node(d, None)
        self.length += 1

    def insert(self, i, d):
        if self.head is None or i == 0:
            self.head = Node(d, self.head)
        else:
            ptr = self.head
            while i > 1 and ptr.next is not None:
                ptr = ptr.next
                i -= 1
            ptr.next = Node(d, ptr.next)
        self.length += 1

    def remove(self, i):  # removes/returns i-th element
        if self.head is None:
            return None
        if i == 0:
            val = self.head.data
            self.head = self.head.next
            self.length -= 1
            return val
        else:
            ptr = self.head
            while i > 1 and ptr.next is not None:
                ptr = ptr.next
                i -= 1
            if i == 1 and ptr.next is not None:
                val = ptr.next.data
                ptr.next = ptr.next.next
                self.length -= 1
                return val
            return None
    def __str__(self):
        values = []
        current = self.head
        while current:
            values.append(str(current.value))  # Convert the node value to string
            current = current.next
        return f"[{', '.join(values)}]"  # Return as a list-like string
        
    def get_elements(self):
        # Return a list of all values in the LinkedList
        elements = []
        current = self.head
        while current:
            elements.append(current.value)
            current = current.next
        return elements

In [3]:
class NetworkWithUndo:
    def __init__(self, N):
        # already implemented
        self.inArray = ArrayListWithUndo()
        for _ in range(N): 
            self.inArray.append(-1)
        self.undos = Stack()
        self.undos.push(N)

    def getSize(self):
        return self.inArray.length()

    def add(self):
        self.inArray.append(-1)
        self.undos.push(1) 

    def root(self, i):
        if i < 0 or i >= len(self.inArray):
            raise IndexError(f"Node {i} is out of bounds.")


        visited = []
        while self.inArray[i] >= 0:
            visited.append(i)
            i = self.inArray[i]
        
        # Path compression
        for node in visited:
            self.inArray.set(node,i)
            self.undos.push(1)
            return i

    def merge(self, i, j):
        #if self.inArray.get(i) <= 0 and self.inArray.get(j) <= 0:
           #assert(0)

        if self.inArray.get(i) < self.inArray.get(j):
            self.inArray.set(i, self.inArray.get(i) + self.inArray.get(j))
            self.inArray.set(j, i)
        else:
            self.inArray.set(j, self.inArray.get(i) + self.inArray.get(j))
            self.inArray.set(i, j)
        self.undos.push(2)

    def undo(self):
        if self.undos.size == 0:
            return

        last_op = self.undos.pop()

        for x in range(last_op):
            self.inArray.undo()
            pass

    def toArray(self):
        return self.inArray.toArray()
        
    def __str__(self):
        return str(self.toArray()) + "\n-> " + str(self.undos)


In [4]:
# minimal tests NetworkWithUndo

def tprint(n,i,s=""):
    print("\n=== Test",i,"===")
    if s!="": print(s)
    print(n,"\nsize",n.getSize())

net = NetworkWithUndo(9)
tprint(net,0)
net.merge(4,0); net.merge(0,3); net.merge(2,1); net.merge(6,8); net.merge(8,7)
tprint(net,1)
net.merge(0,8)
tprint(net,2)
x = net.root(4)
tprint(net,3,"root of 4: "+str(x))
net.undo()
tprint(net,4)
net.undo()
tprint(net,5)
net.merge(0,1)
tprint(net,6)
for i in range(8): 
    net.undo(); tprint(net,i+6)


=== Test 0 ===
[-1, -1, -1, -1, -1, -1, -1, -1, -1]
-> [9] 
size 9

=== Test 1 ===
[-3, -2, 1, 0, 0, -1, 8, 8, -3]
-> [2, 2, 2, 2, 2, 9] 
size 9

=== Test 2 ===
[8, -2, 1, 0, 0, -1, 8, 8, -6]
-> [2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 3 ===
root of 4: 8
[8, -2, 1, 0, 8, -1, 8, 8, -6]
-> [1, 2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 4 ===
[8, -2, 1, 0, 0, -1, 8, 8, -6]
-> [2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 5 ===
[-3, -2, 1, 0, 0, -1, 8, 8, -3]
-> [2, 2, 2, 2, 2, 9] 
size 9

=== Test 6 ===
[-5, 0, 1, 0, 0, -1, 8, 8, -3]
-> [2, 2, 2, 2, 2, 2, 9] 
size 9

=== Test 6 ===
[-3, -2, 1, 0, 0, -1, 8, 8, -3]
-> [2, 2, 2, 2, 2, 9] 
size 9

=== Test 7 ===
[-3, -2, 1, 0, 0, -1, 8, -1, -2]
-> [2, 2, 2, 2, 9] 
size 9

=== Test 8 ===
[-3, -2, 1, 0, 0, -1, -1, -1, -1]
-> [2, 2, 2, 9] 
size 9

=== Test 9 ===
[-3, -1, -1, 0, 0, -1, -1, -1, -1]
-> [2, 2, 9] 
size 9

=== Test 10 ===
[-2, -1, -1, -1, 0, -1, -1, -1, -1]
-> [2, 9] 
size 9

=== Test 11 ===
[-1, -1, -1, -1, -1, -1, -1, -1, -1]
-> [9] 
size 9


In [7]:
class Gadget:
    def __init__(self):
        # already implemented
        self.inNetwork = NetworkWithUndo(0)
        self.subsize = 0
        self.nameMap = {}
        self.undos = Stack()
        self.helper = None # you can edit this line
        
    def getSize(self):
        # already implemented
        return self.inNetwork.getSize()
        
    def isIn(self, name):
        # already implemented
        return name in self.nameMap
        
    def add(self, name):
        if name in self.nameMap:
            self.undos.push(("oth", 0, None))  # No changes needed; just track.
            return

        # Assign a new index to the name and add it to the network.
        node_id = len(self.nameMap)
        self.nameMap[name] = node_id
        self.helper[name] = len(self.undos.push)  # Track step count for `clean`.
        self.inNetwork.add()
        self.subsize += 1

        # Record this operation for undo.
        self.undos.push(("rem", 1, name))

    def subnets(self):
        # Retrieve connected components from the underlying network.
        n = self.inNetwork.getSize()
        root_map = {}
        subnets = {}

        for node, idx in self.nameMap.items():
            root = self.inNetwork.root(idx)
            root_map[node] = root

        # Group nodes by their root.
        for node, root in root_map.items():
            if root not in subnets:
                subnets[root] = []
            subnets[root].append(node)

        return list(subnets.values())

    def connect(self, name1, name2):
        if name1 not in self.nameMap or name2 not in self.nameMap:
            raise ValueError("Both nodes must exist in the gadget.")

        idx1 = self.nameMap[name1]
        idx2 = self.nameMap[name2]

        if self.inNetwork.root(idx1) == self.inNetwork.root(idx2):
            return True  # Already connected.

        # Merge clusters in the network and adjust the subnet count.
        self.inNetwork.merge(idx1, idx2)
        self.subsize -= 1

        # Record this operation for undo.
        self.undos.push(("brk", 1, None))
        return False

    def clean(self, name):
        if name not in self.helper:
            raise ValueError("Node not found in the gadget.")

        # Find how many steps back this node was added.
        steps = self.helper[name]
        self.undo(steps)

    def undo(self, n):
        while n > 0 and self.undos.size > 0:
            op, steps, name = self.undos.pop()

            if op == "rem":
                # Remove the node from the gadget.
                del self.nameMap[name]
                del self.helper[name]
                self.subsize -= 1
            elif op == "brk":
                # Increase the subnet count.
                self.subsize += 1

            # Revert the underlying network changes.
            self.inNetwork.undo(steps)
            n -= 1

        # If all operations are undone, reset to an empty state.
        if not self.nameMap:
            self.subsize = 0
            self.helper.clear()
        
    def toArray(self):
        # already implemented
        A = self.inNetwork.toArray()
        for s in self.nameMap:
            i = self.nameMap[s]
            A[i] = (s,A[i])
        return A
        
    def __str__(self):
        # already implemented
        return str(self.toArray())+"\n-> "+str(self.nameMap)+"\n-> "+str(self.undos)

In [8]:
# minimal tests Gadget

def tprint(g,i,s=""):
    print("\n=== Test",i,"===")
    if s!="": print(s)
    print(g,"\nsize",g.getSize(),"subsize",g.subsize)

g = Gadget()
A = ["128.0.0.1", "216.58.204.68", "212.58.235.1", "qmul", "Nikos.1", "Nikos.2", "Edon.1", "Shitong.1"]
for x in A: g.add(x)
tprint(g,0)
g.add("Nikos.1")
tprint(g,1)
x = g.connect("Nikos.1","Nikos.2")
tprint(g,2,"connected N.1 N.2: "+str(x))
x = g.connect("Nikos.1","Edon.1")
tprint(g,3,"connected N.1 E.1: "+str(x))
x = g.subnets()
tprint(g,4,"subnets: "+str(x))
g.clean("Nikos.1")
tprint(g,5,"Cleaning Nikos.1")

g.add("Nikos.3")
tprint(g,6,"Add Nikos.3")
A = g.toArray()
for (s,_) in A:
    g.connect("Nikos.3",s)
tprint(g,7,"Connect all to Nikos.3")
x = g.subnets()
tprint(g,8,"subnets: "+str(x))

TypeError: object of type 'method' has no len()